In [1]:
import numpy as np
import pandas as pd
import os

os.chdir("../")
from helpers import create_polling_id

In [2]:
df = pd.read_excel("https://www.servel.cl/wp-content/uploads/2022/12/resultados_elecciones_presidenciales_ce_1989_2017_Chile.xlsx")
df.head()

,Tipo de Elección,Cargo,Fecha de Elección,Año de Elección,Inicio de Período,Fin de Período,Período,Votación Presidencial,Id Región,Región,...,Sexo Mesa,Candidato (a),Nombres,Apellido Paterno,Apellido Materno,Sexo,Electo(a),Partido,Sigla Partido,Votos Totales
0,PRESIDENCIAL,PRESIDENTE,1989-12-14,1989,1990,1994,1990 a 1994,UNICA VOTACIÓN,1,DE TARAPACA,...,HOMBRE,FRANCISCO JAVIER ERRAZURIZ,FRANCISCO JAVIER,ERRAZURIZ,TALAVERA,HOMBRE,NaN,INDEPENDIENTE,INDEP,7065
1,PRESIDENCIAL,PRESIDENTE,1989-12-14,1989,1990,1994,1990 a 1994,UNICA VOTACIÓN,1,DE TARAPACA,...,HOMBRE,HERNAN BUCHI,HERNAN,BUCHI,BUC,HOMBRE,NaN,INDEPENDIENTE,INDEP,11549
2,PRESIDENCIAL,PRESIDENTE,1989-12-14,1989,1990,1994,1990 a 1994,UNICA VOTACIÓN,1,DE TARAPACA,...,HOMBRE,PATRICIO AYLWIN,PATRICIO,AYLWIN,AZOCAR,HOMBRE,SI,DEMOCRATA CRISTIANO,DC,23230
3,PRESIDENCIAL,PRESIDENTE,1989-12-14,1989,1990,1994,1990 a 1994,UNICA VOTACIÓN,1,DE TARAPACA,...,HOMBRE,VOTOS NULOS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,561
4,PRESIDENCIAL,PRESIDENTE,1989-12-14,1989,1990,1994,1990 a 1994,UNICA VOTACIÓN,1,DE TARAPACA,...,HOMBRE,VOTOS EN BLANCO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,392


In [3]:
columns = {
    "Tipo de Elección": "election_type",
    "Cargo": "position",
    "Fecha de Elección": "date",
    "Año de Elección": "year",
    "Inicio de Período": "starting_period",
    "Fin de Período": "ending_period",
    "Período": "period",
    "Votación Presidencial": "election_round",
    "Id Región": "region_id",
    "Región": "region",
    "Provincia": "province_id",
    "Nombre Provincia": "province",
    "Comuna": "commune",
    "Circunscripción Electoral": "circunscription",
    "Sexo Mesa": "polling_station_sex",
    "Candidato (a)": "candidate",
    "Nombres": "firstname",
    "Apellido Paterno": "lastname",
    "Apellido Materno": "lastname_2",
    "Sexo": "sex",
    "Electo(a)": "elected",
    "Partido": "party",
    "Sigla Partido": "party_acronym",
    "Votos Totales": "value"
}

df = df.rename(columns=columns)
df.head()

,election_type,position,date,year,starting_period,ending_period,period,election_round,region_id,region,...,polling_station_sex,candidate,firstname,lastname,lastname_2,sex,elected,party,party_acronym,value
0,PRESIDENCIAL,PRESIDENTE,1989-12-14,1989,1990,1994,1990 a 1994,UNICA VOTACIÓN,1,DE TARAPACA,...,HOMBRE,FRANCISCO JAVIER ERRAZURIZ,FRANCISCO JAVIER,ERRAZURIZ,TALAVERA,HOMBRE,NaN,INDEPENDIENTE,INDEP,7065
1,PRESIDENCIAL,PRESIDENTE,1989-12-14,1989,1990,1994,1990 a 1994,UNICA VOTACIÓN,1,DE TARAPACA,...,HOMBRE,HERNAN BUCHI,HERNAN,BUCHI,BUC,HOMBRE,NaN,INDEPENDIENTE,INDEP,11549
2,PRESIDENCIAL,PRESIDENTE,1989-12-14,1989,1990,1994,1990 a 1994,UNICA VOTACIÓN,1,DE TARAPACA,...,HOMBRE,PATRICIO AYLWIN,PATRICIO,AYLWIN,AZOCAR,HOMBRE,SI,DEMOCRATA CRISTIANO,DC,23230
3,PRESIDENCIAL,PRESIDENTE,1989-12-14,1989,1990,1994,1990 a 1994,UNICA VOTACIÓN,1,DE TARAPACA,...,HOMBRE,VOTOS NULOS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,561
4,PRESIDENCIAL,PRESIDENTE,1989-12-14,1989,1990,1994,1990 a 1994,UNICA VOTACIÓN,1,DE TARAPACA,...,HOMBRE,VOTOS EN BLANCO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,392


In [4]:
election_round = "first_round"
year = 2017

rnd = ["UNICA VOTACIÓN", "PRIMERA VOTACIÓN"] if election_round == "first_round" else ["SEGUNDA VOTACIÓN"]

In [5]:
df["polling_id"] = create_polling_id(df, ["region", "commune", "circunscription", "polling_station_sex"])
df["candidate"] = df["candidate"].str.strip()

#### Spoilt votes

In [6]:
df_spoilt = df[df["candidate"].isin(["VOTOS NULOS", "VOTOS EN BLANCO"])].copy()
df_spoilt = df_spoilt[["candidate", "value", "polling_id"]]
df_spoilt["candidate"] = df_spoilt["candidate"].replace({"VOTOS NULOS": "SPOILT", "VOTOS EN BLANCO": "BLANK"})
df_spoilt["flag_candidates"] = 0
df_spoilt.head()

,candidate,value,polling_id,flag_candidates
3,SPOILT,561,DE TARAPACA-ARICA-ARICA-HOMBRE,0
4,BLANK,392,DE TARAPACA-ARICA-ARICA-HOMBRE,0
8,SPOILT,1154,DE TARAPACA-ARICA-ARICA-MUJER,0
9,BLANK,599,DE TARAPACA-ARICA-ARICA-MUJER,0
13,SPOILT,0,DE TARAPACA-CAMARONES-CAMARONES-HOMBRE,0


In [7]:
df = df[(df["year"] == year) & (df["election_round"].isin(rnd))]
df = df[~df["candidate"].isin(["VOTOS NULOS", "VOTOS EN BLANCO", "TOTALES", np.nan])]

In [8]:
df.candidate.unique()

array(['ALFREDO SFEIR YOUNIS', 'EVELYN MATTHEI FORNET',
       'FRANCO ALDO PARISI FERNANDEZ', 'MARCEL CLAUDE REYES',
       'MARCO ENRIQUEZ-OMINAMI GUMUCIO', 'MICHELLE BACHELET JERIA',
       'RICARDO ISRAEL ZIPPER', 'ROXANA MIRANDA MENESES',
       'TOMAS JOCELYN-HOLT LETELIER'], dtype=object)

In [9]:
df_filtered = df.groupby(["polling_id", "candidate", "party"]).agg({"value": "sum"})
df_filtered["rate"] = df_filtered.groupby(level=[0], group_keys=False).apply(lambda x: x/x.sum())
df_filtered = df_filtered.reset_index()
df_filtered["rank"] = df_filtered.groupby(["polling_id"])["value"].rank("min", ascending=False).astype(int)
df_filtered["flag_candidates"] = 1
df_filtered.head()

,polling_id,candidate,party,value,rate,rank,flag_candidates
0,AISEN DEL GRAL. CARLOS IBAÑEZ-AISEN-AISEN-MIXTA,ALFREDO SFEIR YOUNIS,PARTIDO ECOLOGISTA VERDE,56,0.007698,7,1
1,AISEN DEL GRAL. CARLOS IBAÑEZ-AISEN-AISEN-MIXTA,EVELYN MATTHEI FORNET,UNION DEMOCRATA INDEPENDIENTE,1336,0.183643,2,1
2,AISEN DEL GRAL. CARLOS IBAÑEZ-AISEN-AISEN-MIXTA,FRANCO ALDO PARISI FERNANDEZ,INDEPENDIENTE,1334,0.183368,3,1
3,AISEN DEL GRAL. CARLOS IBAÑEZ-AISEN-AISEN-MIXTA,MARCEL CLAUDE REYES,PARTIDO HUMANISTA,149,0.020481,5,1
4,AISEN DEL GRAL. CARLOS IBAÑEZ-AISEN-AISEN-MIXTA,MARCO ENRIQUEZ-OMINAMI GUMUCIO,PARTIDO PROGRESISTA,419,0.057595,4,1


In [10]:
data = pd.concat([
    df_filtered,
    df_spoilt
])

data.head()

,polling_id,candidate,party,value,rate,rank,flag_candidates
0,AISEN DEL GRAL. CARLOS IBAÑEZ-AISEN-AISEN-MIXTA,ALFREDO SFEIR YOUNIS,PARTIDO ECOLOGISTA VERDE,56,0.007698,7.0,1
1,AISEN DEL GRAL. CARLOS IBAÑEZ-AISEN-AISEN-MIXTA,EVELYN MATTHEI FORNET,UNION DEMOCRATA INDEPENDIENTE,1336,0.183643,2.0,1
2,AISEN DEL GRAL. CARLOS IBAÑEZ-AISEN-AISEN-MIXTA,FRANCO ALDO PARISI FERNANDEZ,INDEPENDIENTE,1334,0.183368,3.0,1
3,AISEN DEL GRAL. CARLOS IBAÑEZ-AISEN-AISEN-MIXTA,MARCEL CLAUDE REYES,PARTIDO HUMANISTA,149,0.020481,5.0,1
4,AISEN DEL GRAL. CARLOS IBAÑEZ-AISEN-AISEN-MIXTA,MARCO ENRIQUEZ-OMINAMI GUMUCIO,PARTIDO PROGRESISTA,419,0.057595,4.0,1


In [11]:
print(df_filtered.shape)
df_filtered.head()

(5148, 7)


,polling_id,candidate,party,value,rate,rank,flag_candidates
0,AISEN DEL GRAL. CARLOS IBAÑEZ-AISEN-AISEN-MIXTA,ALFREDO SFEIR YOUNIS,PARTIDO ECOLOGISTA VERDE,56,0.007698,7,1
1,AISEN DEL GRAL. CARLOS IBAÑEZ-AISEN-AISEN-MIXTA,EVELYN MATTHEI FORNET,UNION DEMOCRATA INDEPENDIENTE,1336,0.183643,2,1
2,AISEN DEL GRAL. CARLOS IBAÑEZ-AISEN-AISEN-MIXTA,FRANCO ALDO PARISI FERNANDEZ,INDEPENDIENTE,1334,0.183368,3,1
3,AISEN DEL GRAL. CARLOS IBAÑEZ-AISEN-AISEN-MIXTA,MARCEL CLAUDE REYES,PARTIDO HUMANISTA,149,0.020481,5,1
4,AISEN DEL GRAL. CARLOS IBAÑEZ-AISEN-AISEN-MIXTA,MARCO ENRIQUEZ-OMINAMI GUMUCIO,PARTIDO PROGRESISTA,419,0.057595,4,1


In [12]:
data.to_csv(f"../data_output/Chile/{year}_{election_round}_circunscription.csv.gz", compression="gzip", index=False)

In [13]:
df_location = df[["polling_id", "region_id", "region", "province", "commune", "circunscription", "polling_station_sex"]].drop_duplicates()
df_location.to_csv(f"../data_output/Chile/{year}_{election_round}_circunscription_location.csv.gz", compression="gzip", index=False)